In [19]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 
#!import banking/ContaCorrente.cs 
#!import banking/Boleto.cs 
#!import banking/BankFunctions.cs 
#!import functioncalling/FunctionCalling.cs 


// Import Semantic Kernel
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.7"
using Azure.AI.OpenAI;
var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();


var endpoint = new Uri(azureEndpoint);
var credentials = new Azure.AzureKeyCredential(apiKey);
var openAIClient = new OpenAIClient(endpoint, credentials);

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.7

In [20]:
var system = 
""" 
Voce é um assistente bancário que ajuda um cliente a transacionar em usa conta. Seja solicito e educado.
""";

In [21]:
var completionOptions = new ChatCompletionsOptions
{
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f, // Top P
};

In [22]:
completionOptions.Messages.Add(new ChatMessage(ChatRole.System, system));

completionOptions.Messages

[ Azure.AI.OpenAI.ChatMessage ] IsUndefined False Count 1 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.ChatMessage Role system Content Voce é um assistente bancário que ajuda um cliente a transacionar em usa conta. Seja solicito e educado. Name <null> FunctionCall <null> AzureExtensionsContext <null>

In [23]:
FunctionCalling caller = new FunctionCalling();

var contaCorrentPrincipal = new ContaCorrente("123");
contaCorrentPrincipal.Depositar(1000);

Dictionary<string, ContaCorrente> contasCorrentes = new Dictionary<string, ContaCorrente>() {
    { "André", new ContaCorrente("456") },
    { "João", new ContaCorrente("789") }
};


Boleto[] boletos = { Boleto.Criar("1", 450), Boleto.Criar("2", 150) , Boleto.Criar("3", 1500) };

BankFunctions bf = new BankFunctions(contaCorrentPrincipal, boletos, contasCorrentes);
bf.ToString()

Conta Corrente Principal - Saldo: 1000
Boletos Pendentes: 
  - Numero:1 - Valor:450 - Pago:False
  - Numero:2 - Valor:150 - Pago:False
  - Numero:3 - Valor:1500 - Pago:False
Contas Relacionadas: 
  - Nome: André - Numero:456 - Saldo:0
  - Nome: João - Numero:789 - Saldo:0


In [24]:
var funDef = caller.GetDefinitionsFromObject(bf);
foreach (var fd in funDef) {
    completionOptions.Functions.Add(fd);
}

funDef

index value 0 Azure.AI.OpenAI.FunctionDefinition Name Saldo Description Obtem o Saldo da Conta corrente do usuario atual Parameters { "type": "object", "properties": {}} 1 Azure.AI.OpenAI.FunctionDefinition Name ObtemBoletosPendentes Description Obtem os boletos pendentes do usuario atual Parameters { "type": "object", "properties": {}} 2 Azure.AI.OpenAI.FunctionDefinition Name PagarBoleto Description Obtem os boletos pendentes do usuario atual Parameters { "type": "object", "properties": {"numeroBoleto":{"Type":"string","Description":"Numero do Boleto"}}} 3 Azure.AI.OpenAI.FunctionDefinition Name ObterContaRelacionada Description Obtem os numeros de contas relacionadas ao usuario atual pelo nome ou apelido Parameters { "type": "object", "properties": {"nomeConta":{"Type":"string","Description":"Nome da Conta"}}} 4 Azure.AI.OpenAI.FunctionDefinition Name Transferir Description Transferir um valor para uma conta relacionada Parameters { "type": "object", "properties": {"numeroConta":{"Type":"string","Description":"Numero da Conta relacionada"}, "valor":{"Type":"string","Description":"Valor a ser transferido"}}} 5 Azure.AI.OpenAI.FunctionDefinition Name ToString Description <null> Parameters { "type": "object", "properties": {}}

In [25]:
completionOptions.Messages.Add(new ChatMessage(ChatRole.User, "Pague todos os boletos da minha conta e transfira 100 para o João. Me diga o saldo."));


In [26]:
do {
var response = await openAIClient.GetChatCompletionsAsync(
            model,
            completionOptions);

var responseChoice = response.Value.Choices[0];
 if (responseChoice.FinishReason == CompletionsFinishReason.FunctionCall)
{
            // Include the FunctionCall message in the conversation history
            completionOptions.Messages.Add(responseChoice.Message);
            var funcResp = caller.ExecuteFunction(responseChoice.Message.FunctionCall, bf);
            completionOptions.Messages.Add(new ChatMessage() { Role = ChatRole.Function, Content = funcResp, Name=responseChoice.Message.FunctionCall.Name});
} else {
    completionOptions.Messages.Add(responseChoice.Message);
    Console.WriteLine(responseChoice.Message.Content);
    break;
}

}while(true);
completionOptions.Messages


ExecuteFunction ObtemBoletosPendentes called
Arguments: {}
Boletos Pendentes: 
Numero:1 - Valor:450
Numero:2 - Valor:150
Numero:3 - Valor:1500

ExecuteFunction PagarBoleto called
Arguments: {
"numeroBoleto": 1
}
Parameter numeroBoleto of type String
Element: 1
Pagar Boleto 1 chamado
ExecuteFunction PagarBoleto called
Arguments: {
"numeroBoleto": 2
}
Parameter numeroBoleto of type String
Element: 2
Pagar Boleto 2 chamado
ExecuteFunction PagarBoleto called
Arguments: {
"numeroBoleto": 3
}
Parameter numeroBoleto of type String
Element: 3
Pagar Boleto 3 chamado
ExecuteFunction ObterContaRelacionada called
Arguments: {
"nomeConta": "João"
}
Parameter nomeConta of type String
Element: João
ObterContaRelacionada João chamado
ExecuteFunction Transferir called
Arguments: {
"numeroConta": 789,
"valor": 100
}
Parameter numeroConta of type String
Element: 789
Parameter valor of type String
Element: 100
Transferir 100 para 789 chamado
ExecuteFunction Saldo called
Arguments: {}
Saldo: -1200
Todos os

[ Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage, Azure.AI.OpenAI.ChatMessage ] IsUndefined False Count 17 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.ChatMessage Role system Content Voce é um assistente bancário que ajuda um cliente a transacionar em usa conta. Seja solicito e educado. Name <null> FunctionCall <null> AzureExtensionsContext <null> 1 Azure.AI.OpenAI.ChatMessage Role user Content Pague todos os boletos da minha conta e transfira 100 para o João. Me diga o saldo. Name <null> FunctionCall <null> AzureExtensionsContext <null> 2 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name ObtemBoletosPendentes Arguments {} AzureExtensionsContext <null> 3 Azure.AI.OpenAI.ChatMessage Role function Content "Boletos Pendentes: \nNumero:1 - Valor:450\nNumero:2 - Valor:150\nNumero:3 - Valor:1500\n" Name ObtemBoletosPendentes FunctionCall <null> AzureExtensionsContext <null> 4 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name PagarBoleto Arguments {
"numeroBoleto": 1
} AzureExtensionsContext <null> 5 Azure.AI.OpenAI.ChatMessage Role function Content "Boleto 1 pago com sucesso!" Name PagarBoleto FunctionCall <null> AzureExtensionsContext <null> 6 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name PagarBoleto Arguments {
"numeroBoleto": 2
} AzureExtensionsContext <null> 7 Azure.AI.OpenAI.ChatMessage Role function Content "Boleto 2 pago com sucesso!" Name PagarBoleto FunctionCall <null> AzureExtensionsContext <null> 8 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name PagarBoleto Arguments {
"numeroBoleto": 3
} AzureExtensionsContext <null> 9 Azure.AI.OpenAI.ChatMessage Role function Content "Boleto 3 pago com sucesso!" Name PagarBoleto FunctionCall <null> AzureExtensionsContext <null> 10 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name ObterContaRelacionada Arguments {
"nomeConta": "João"
} AzureExtensionsContext <null> 11 Azure.AI.OpenAI.ChatMessage Role function Content "Conta Jo\u00E3o encontrada! N\u00FAmero da Conta: 789" Name ObterContaRelacionada FunctionCall <null> AzureExtensionsContext <null> 12 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name Transferir Arguments {
"numeroConta": 789,
"valor": 100
} AzureExtensionsContext <null> 13 Azure.AI.OpenAI.ChatMessage Role function Content "Transferencia de 100 para 789 realizada com sucesso!" Name Transferir FunctionCall <null> AzureExtensionsContext <null> 14 Azure.AI.OpenAI.ChatMessage Role assistant Content <null> Name <null> FunctionCall Azure.AI.OpenAI.FunctionCall Name Saldo Arguments {} AzureExtensionsContext <null> 15 Azure.AI.OpenAI.ChatMessage Role function Content "-1200" Name Saldo FunctionCall <null> AzureExtensionsContext <null> 16 Azure.AI.OpenAI.ChatMessage Role assistant Content Todos os seus boletos foram pagos e a transferência de 100 para João foi realizada com sucesso. Porém, observe que seu saldo atual é negativo, estando em -1200. Por favor, certifique-se de adicionar fundos à sua conta para evitar quaisquer taxas de saldo negativo. Se precisar de mais alguma ajuda, estou à disposição! Name <null> FunctionCall <null> AzureExtensionsContext <null>

In [27]:
bf.ToString()

Conta Corrente Principal - Saldo: -1200
Boletos Pendentes: 
  - Numero:1 - Valor:450 - Pago:True
  - Numero:2 - Valor:150 - Pago:True
  - Numero:3 - Valor:1500 - Pago:True
Contas Relacionadas: 
  - Nome: André - Numero:456 - Saldo:0
  - Nome: João - Numero:789 - Saldo:100
